## Part 7. Aristotle prover

Harmonic (the company behind Newclid) also has a Lean-based prover called Aristotle that's their closed source flagship product. The company provides a web interface and API at https://aristotle.harmonic.fun that anyone can register and try (caveat it is very slow with long queue for job requests). This note uses our small, friendly geometry problem to demonstrate the workflow, while the same system is aimed at harder, cutting-edge problems across mathematics (geometry, number theory, analysis, and more) [ARISTOTLE].

**How Aristotle differs from Newclid (short version).** Newclid is a geometry-first system: you give a diagram-style construction and goals, and its symbolic engine tries to derive a synthetic proof inside that geometry domain. Aristotle is a general Lean prover: it formalizes statements in Lean and then proves them with Lean tactics, so its output is a formal Lean proof rather than a geometry-specific trace. For geometry problems, Aristotle's geometric solver can still use geometric reasoning (which appeared to be Newclid like), but in this demo it chooses a coordinate model, which looks closer to the analytic approach in Part 2. This is why the artifacts here feel more Lean/coordinate-heavy than Newclid's geometric narrative.

Harmonic reports that Aristotle achieved gold medal-level performance on the 2025 International Mathematical Olympiad (IMO) solving 5 out of 6 problems, with formally verified solutions[HN-IMO][BW-IMO]. In the problem solved, problem 2 is about pure plane geometry [IMO2025P2]. It is vague if the solution adopted used Newclid approach, which Aristotle paper hinted that a separate solver is indeed used for geometry problems, following by the paper describing Newclid a month later. The proof also appears to be a geometry deduction instead of the Lean language.

In general those IMO announcements are a useful backdrop for this demo: we stay with our simple trapezoid ratio to show how aristotle solve such simple problems in geometry, but the tooling is designed for much harder proofs in various domain beyond geometry. Without the full knowledge on how Aristotle's geometric engine differs from the open sourced Newclid, we will focus on how the solution to our simple polygon problem differ between the two to give an observation in this part of the notes. In general the aforementioned white papers propose that Aristotle trained a LLM with reinforcement learning to handle highly parallel Monte Carlo Graph Search over Lean proof states, with policy/value network to pick tree nodes and estimate promise. This differs from the Newclid default BSF tree search, which is enhanced by auxiliary point helper when the search logic stuck. This part of the design is beyond the scope of the current discussion. For our simple problem Aristotle indeed gives a distinct solution repying on Lean.

Aristotle has been aimed and used beyond contest problems. In late 2025, it produced a Lean proof of a simplified variant of Erdos Problem 124. Chinese coverage highlighted that the system completed a variant autonomously in about six hours, while other reports noted that the solved statement was easier than the original conjecture [E124-6H]. Nonetheless this shows its early impact to the math community.

A few days later, Harmonic's tool was credited for helping resolve Erdos Problem 1026. A December 14, 2025 write-up described how the AI tool proved a key step $c(k^2)=1/k$ in Lean as part of the eventual solution [E1026-48H]. Harmonic's own news page claims that since Aristotle's public launch in late 2025, mathematicians have used it to formalize proofs across levels and that the company sees this as a new era of discovery. [HN-MSI]

For broader context on Erdos problems, the community-maintained erdosproblems GitHub database tracks problem status, prizes, formalizations, and links to related resources. It is essentially a living index of the current state of the problem list and related metadata, with an auto-generated table based on the repository's data files [ERDOS-SOTA]. Top mathematician Prof. Torrence Tao has been pioneering this efforts and other promotion of using AI for math research. The accompanying wiki page on AI contributions is more careful and nuanced: it catalogs where AI tools have been used, but also stresses that this is not a benchmark or leaderboard, that literature review can be incomplete, and that assessments are provisional and context-dependent. The page is best read as a reference for tracing claims and understanding the role AI has actually played [ERDOS-AI].
<div align="center">
<img alt="Aristotle prompt interface" src="aristotle_prompt1.png" style="height:320px; width:auto; display:block; margin:0 auto;">

*Figure 7.1: Aristotle's prompt interface. It accepts formal Lean input as well as informal text or file uploads.*
</div>
As Figure 7.1 shows, the command line application of Aristotle can offer an interactive proof interface. Also they offer an API interface in python which is used in the code below to solve our polygon problem, including both questions. Note that the problem text is written in informal plain English, which is accepted by their API as well as the prompt, aside from the alternative lean input format. Under the hood the model clearly utilize strong LLM for understanding such descriptions. In fact the team has described its solver architecture as a pretrained LLM further trained with reinforcement learning for reasoning [ARISTOTLE].

---

In [ ]:
import os
import logging
from pathlib import Path
import os
import re

def load_aristotle_api_key():
    key_file = Path.home() / ".aristotle_api_key"
    text = key_file.read_text(encoding="utf-8")

    m = re.search(r'ARISTOTLE_API_KEY="([^"]+)"', text)
    if not m:
        raise RuntimeError("ARISTOTLE_API_KEY not found in ~/.aristotle_api_key")
    return m.group(1)

os.environ["ARISTOTLE_API_KEY"] = load_aristotle_api_key()
print("ARISTOTLE_API_KEY loaded into this kernel")

import aristotlelib
from aristotlelib.project import Project, ProjectInputType

async def prove_polygon_ratio():
    api_key = os.getenv("ARISTOTLE_API_KEY")
    if api_key:
        aristotlelib.set_api_key(api_key)
    else:
        raise RuntimeError("ARISTOTLE_API_KEY environment variable not set")

    logging.basicConfig(level=logging.INFO, format="%(levelname)s - %(message)s")

    problem_text = """
Polygon ratio problem (informal statement).
In trapezoid ABCD, AD ∥ BC, ∠C = 90°, and BC = CD.
Point E lies on CD with DE = AD. Through E draw EF ⟂ AB with F the foot.
Draw BD; it meets AE and EF at G and H, respectively. Prove:
1) BF * HE = GE * BH.
2) BE * DH = BD * EH.
""".strip() + "\n"

    await Project.prove_from_file(
        input_content=problem_text,
        output_file_path="aristotle/polygon_ratio_problem_aristotle.lean",
        validate_lean_project=False,
        project_input_type=ProjectInputType.INFORMAL,
        auto_add_imports=False,
    )

await prove_polygon_ratio()


<div align="center">
<img alt="Aristotle web interface screenshot" src="aristotle_harmonic.fun_output.png" style="height:320px; width:auto; display:block; margin:0 auto;">

*Figure 7.2 The Aristotle web UI at harmonic.fun. The demo job below was queued for a while and then ran for ~20 minutes, suggesting a resource constrained run for the trial account possibly not even on the GPU.*
</div>


### Lean quick intro

Lean is an interactive theorem prover: humans (or automated tactics) provide the proof steps, and Lean checks every step for correctness. In typical use, the proof is supplied by the user via tactics or proof terms, though automation can discharge many routine goals.

Before Lean became popular, prominent proof assistants and formal verification systems included Coq, Isabelle/HOL, HOL Light, ACL2, and Mizar.  Lean was created at Microsoft Research by Leonardo de Moura and collaborators. It began as a general-purpose theorem prover for formal verification and mathematics, and later grew a large community library (mathlib). Rough timeline (high-level): early 2010s initial Lean releases; mid2010s Lean 3 matured and became the main community platform; early 2020s Lean 4 modernized the system and tooling.

In recent years, researchers and tool builders have started using Lean not just for verification but also for *assisting* proof discovery: automated tactics, proof search, and AI-guided systems propose steps that Lean then verifies. This notebook uses Aristotle to generate a Lean proof automatically, while Lean itself remains the verifier. [LEAN]


### Proof 7.1 Aristotle (Lean files + reading guide)

> #### Human-readable summary (abstracted based on Lean)
> Aristotle proves the ratio identities by coordinatizing the trapezoid in the complex plane. It sets
> $C=(0,0)$, $B=(a,0)$, $D=(0,a)$, $A=(x,a)$, and $E=(0,a-x)$,
> so the geometric constraints are encoded directly. The auxiliary points $F,G,H$ are then defined algebraically (projection and intersections). The two target identities are reduced to equalities of squared distances, denominators are cleared, and the resulting expressions are simplified by algebraic tactics. In short: the geometry is converted to algebra, and Lean verifies the algebraic identities. This translation indicated the step of converting the informal input description to a format one saving some work from the human.
>
>
> #### Human-readable analytical proof (abstracted based on Lean)
> 1) **Coordinate model.** Set $C=(0,0)$, $B=(a,0)$, $D=(0,a)$, $A=(x,a)$, $E=(0,a-x)$. This makes $AD\parallel BC$, $\angle C=90^\circ$, $BC=CD$, and $DE=AD$ automatic.
>
> 2) **How $H$ is found.** The diagonal $BD$ has direction $(-a,\,a)$ and can be parametrized as $(a\,(1-s),\,a\,s)$. The line through $E$ perpendicular to $AB$ has direction $(a,\,-(x-a))$ and can be parametrized as $(0,a-x)+u\,(a,\,-(x-a))$. Solving
> $u a=a\,(1-s)$ and $a-x-u(x-a)=a\,s$ gives $u=\tfrac{x}{2a-x}$, hence
> $$
> H=\Big(\frac{ax}{2a-x},\,\frac{2a(a-x)}{2a-x}\Big).
> $$
>
> 3) **How $F$ is found.** $F$ is the orthogonal projection of $E$ onto the line $AB$. In coordinates this uses the standard projection formula
> $$
> F=B+t(A-B),\quad t=\frac{\langle E-B,\,A-B\rangle}{\langle A-B,\,A-B\rangle}.
> $$
> This yields explicit coordinates for $F$, which Lean uses algebraically rather than geometrically.
>
> 4) **Second identity.** Compute squared lengths:
> $BE^2=a^2+(a-x)^2$, $BD^2=2a^2$,
> $DH^2=\tfrac{2a^2x^2}{(2a-x)^2}$, and
> $EH^2=\tfrac{x^2(a^2+(a-x)^2)}{(2a-x)^2}$.
> Thus $BE^2\,DH^2=BD^2\,EH^2$, implying $BE\cdot DH=BD\cdot EH$.
>
> 5) **First identity.** The proof is analogous: substitute the explicit $F,G,H$ coordinates, expand squared distances, clear denominators, and simplify. The Lean proof follows the same algebraic pattern as the second identity, so we omit the details.

#### Input file (informal statement) - `@polygon_ratio_problem.lean`
This is the informal problem text sent to Aristotle (comments start with `--`):

```lean
-- Polygon ratio problem (informal statement for Aristotle)
-- In trapezoid ABCD, AD ∥ BC, ∠C = 90°, and BC = CD.
-- Point E lies on CD with DE = AD. Through E draw EF ⟂ AB with F the foot.
-- Draw BD; it meets AE and EF at G and H, respectively. Prove:
-- 1) BF * HE = GE * BH
-- 2) BE * DH = BD * EH
```


#### Aristotle output (formal Lean proof) - `polygon_ratio_problem_aristotle.lean`
Below are key parts of the generated Lean file with **line numbers**, each followed by extended commentary that explains the Lean keywords and how the proof progresses.


**Lean output (lines 1-53):**

```lean
 1: /-
 2: This file was generated by Aristotle.
 3: 
 4: Lean version: leanprover/lean4:v4.24.0
 5: Mathlib version: f897ebcf72cd16f89ab4577d0c826cd14afaafc7
 6: This project request had uuid: defb37a9-102b-470c-8442-c3396eccd211
 7: 
 8: To cite Aristotle, tag @Aristotle-Harmonic on GitHub PRs/issues, and add as co-author to commits:
 9: Co-authored-by: Aristotle (Harmonic) <aristotle-harmonic@harmonic.fun>
10: -/
11: 
12: /-
13: We define the points of the trapezoid in the complex plane and prove the two required geometric identities using algebraic simplification of the distances.
14: The points are defined as follows:
15: - C is the origin (0,0).
16: - B is at (a,0).
17: - D is at (0,a).
18: - A is at (x,a).
19: - E is at (0, a-x).
20: - F is the projection of E onto AB.
21: - G is the intersection of AE and BD.
22: - H is the intersection of EF and BD.
23: 
24: We prove:
25: 1. BF * HE = GE * BH (`polygon_ratio_1`)
26: 2. BE * DH = BD * EH (`polygon_ratio_2`)
27: -/
28: 
29: import Mathlib
30: 
31: set_option linter.mathlibStandardSet false
32: 
33: open scoped BigOperators
34: open scoped Real
35: open scoped Nat
36: open scoped Classical
37: open scoped Pointwise
38: 
39: set_option maxHeartbeats 0
40: set_option maxRecDepth 4000
41: set_option synthInstance.maxHeartbeats 20000
42: set_option synthInstance.maxSize 128
43: 
44: set_option relaxedAutoImplicit false
45: set_option autoImplicit false
46: 
47: noncomputable section
48: 
49: /-
50: Definitions of points A, B, C, D, E, F, G, H in the complex plane for the trapezoid problem.
51: -/
52: open Complex
53: 
```

**Commentary (lines 1-53):** This block is metadata and explanatory prose rather than executable code. The multi-line comment delimiters `/- ... -/` mark documentation. The header records the Lean and mathlib versions, which matter because tactics and lemmas can change across releases. The project UUID identifies the Aristotle run. The next comment describes the coordinate model and names the two target identities, giving the reader a roadmap before formal definitions begin. The `import Mathlib` and `set_option` lines also live in this block: they load the library and tune the proof engine (heartbeats, recursion depth, instance search limits) so algebraic automation can complete. This section moves the proof from narrative intent to a precise plan.

**Lean output (lines 54-76):**

```lean
54: noncomputable def pC : ℂ := 0
55: noncomputable def pD (a : ℝ) : ℂ := I * a
56: noncomputable def pB (a : ℝ) : ℂ := a
57: noncomputable def pA (a x : ℝ) : ℂ := x + I * a
58: noncomputable def pE (a x : ℝ) : ℂ := I * (a - x)
59: 
60: noncomputable def pF (a x : ℝ) : ℂ :=
61:   let AB := pA a x - pB a
62:   let EB := pE a x - pB a
63:   let t := (EB * star AB).re / (AB * star AB).re
64:   pB a + t * AB
65: 
66: noncomputable def pG (a x : ℝ) : ℂ := (x / 2) + I * (a - x / 2)
67: 
68: noncomputable def pH (a x : ℝ) : ℂ :=
69:   let den := 2 * a - x
70:   (a * x / den) + I * (2 * a * (a - x) / den)
71: 
72: /-
73: Prove that BF * HE = GE * BH in the configured trapezoid.
74: -/
75: open Complex Metric
76: 
```

**Commentary (lines 54-76):** This block is the fully translated Lean version of the geometric setup. `noncomputable` and `open Complex` make complex-number geometry available. The `def` keyword introduces named definitions; here `pA`, `pB`, `pC`, `pD`, `pE`, `pF`, `pG`, `pH` are points encoded as complex numbers, with `I` the imaginary unit. `pF` encodes orthogonal projection using complex conjugation (`star`) and real parts (`.re`) as dot products. `pG` and `pH` are explicit intersection formulas. This is the precise coordinate model that the later algebraic proofs use.

**Lean output (lines 77-92):**

```lean
77: theorem polygon_ratio_1 (a x : ℝ) (ha : a ≠ 0) (hx : x ≠ 0) (hden : 2 * a ≠ x) :
78:   dist (pF a x) (pB a) * dist (pE a x) (pH a x) = dist (pG a x) (pE a x) * dist (pH a x) (pB a) := by
79:   unfold pF pE pG pH at *;
80:   unfold pA pB at *;
81:   norm_num [ Complex.dist_eq, Complex.normSq, Complex.norm_def ];
82:   norm_num [ Complex.normSq, Complex.div_re, Complex.div_im ] at *;
83:   field_simp;
84:   rw [ ← Real.sqrt_mul <| by positivity, ← Real.sqrt_mul <| by positivity ];
85:   rw [ ← Real.sqrt_mul <| by positivity, ← Real.sqrt_mul <| by positivity ] ; ring;
86:   grind
87: 
88: /-
89: Prove that BE * DH = BD * EH in the configured trapezoid.
90: -/
91: open Complex Metric
92: 
```

**Commentary (lines 77-92):** `theorem` introduces a proposition to prove. The objective statement is on line 77: it asserts the first ratio identity as an equality of distances. The hypotheses `ha`, `hx`, and `hden` assert non-degeneracy so denominators are safe (e.g., `2a-x ≠ 0`). The proof enters tactic mode with `by`. `unfold` expands definitions, turning geometric objects into explicit algebra. `norm_num` expands norms and distances into algebraic expressions. `field_simp` clears denominators. `rw` is the rewrite tactic: it replaces the current goal using a known equality (here, square-root identities), so expressions are put into a form that `ring` and `grind` can finish. This block advances the proof by reducing geometry to a polynomial identity and closing it mechanically.

**Lean output (lines 93-101):**

```lean
 93: theorem polygon_ratio_2 (a x : ℝ) (ha : a ≠ 0) (hx : x ≠ 0) (hden : 2 * a ≠ x) :
 94:   dist (pE a x) (pB a) * dist (pD a) (pH a x) = dist (pD a) (pB a) * dist (pE a x) (pH a x) := by
 95:   unfold pB pD pE pH;
 96:   norm_num [ Complex.normSq, Complex.norm_def, Complex.dist_eq ];
 97:   norm_cast; norm_num [ Complex.normSq, Complex.div_re, Complex.div_im, ha, hx, hden ] ; ring;
 98:   rw [ ← Real.sqrt_mul, ← Real.sqrt_mul ];
 99:   · grind;
100:   · positivity;
101:   · nlinarith [ sq_nonneg ( a - x ) ]
```

**Commentary (lines 93-101):** `polygon_ratio_2` mirrors the first proof. `norm_cast` resolves coercions between numeric types. `positivity` discharges non-negativity side conditions required for square roots. `nlinarith` solves remaining nonlinear arithmetic constraints. Conceptually, this block repeats the same algebraic pipeline: expand distances, clear denominators, and verify the polynomial equality. It completes the proof of the second identity under the same non-degeneracy assumptions.


**Note on proof style.** In this problem Aristotle proceeds by a coordinate model, so the proof is essentially algebraic and very similar in spirit to Part 2. That choice is not a rule of Aristotle; it is just the strategy selected for this run. In contrast, Newclid (Part 6) is designed to stay in the geometry domain and produce a synthetic-style trace from diagram constraints. So the key difference to keep in mind is not *correctness* but *style of reasoning and output*: Aristotle outputs a Lean proof (general formal system), while Newclid outputs a geometry-specific derivation. For other problems, Aristotle can also use geometric/angle-chasing reasoning, as seen in the IMO 2025 P2 formalization [IMO2025P2], but here it stayed with coordinates for simplicity.



All in all tools like lean MathLib (lean language and thereom database) + Aristotle (LLM with RL training in lean) + ChatGPT 5.2 Pro (and its competitor frontier LLM) have become the most cutting edge AI tools for mathematicians, which has revealed a frontier of realization of mathematical AGI.


### References

- [ARISTOTLE] Harmonic. "Aristotle" (web interface). Summary: Web interface and API access for the Aristotle prover. URL: https://aristotle.harmonic.fun; arxiv link of the paper at https://arxiv.org/pdf/2510.01346
- [E124-6H] 36Kr (2025-11-). "AI solved a 30-year math problem in 6 hours. Terence Tao: ChatGPT and others failed." Summary: Coverage of a simplified Erdős #124 variant solved with Aristotle. URL: https://eu.36kr.com/en/p/3576638922980231
- [E1026-48H] 36Kr (2025-12-14). "Terence Tao is shocked: The “pit” dug by a mathematician in 1975 was filled by AI and netizens around the world in just 48 hours." Summary: Coverage of Erdős #1026 and an Aristotle-assisted Lean step. URL: https://eu.36kr.com/en/p/3596176018898947
- [HN-MSI] Harmonic News (2025). "Harmonic announces mathematician sponsorships to accelerate mathematical superintelligence." Summary: Harmonic’s announcement and claims about Aristotle usage. URL: https://harmonic.fun/news
- [HN-IMO] Harmonic News (2025-07-28). "Aristotle Achieves Gold Medal-Level Performance at the International Mathematical Olympiad". Summary: Harmonic announcement of IMO 2025 results and formal verification. URL: https://harmonic.fun/news
- [BW-IMO] Business Wire (2025-07-28). "Harmonic Announces IMO Gold Medal-Level Performance & Launch of First Mathematical Superintelligence (MSI) AI App." Summary: Press release on IMO 2025 result and Aristotle launch. URL: https://www.businesswire.com/news/home/20250728394917/en/Harmonic-Announces-IMO-Gold-Medal-Level-Performance-Launch-of-First-Mathematical-Superintelligence-MSI-AI-App
- [IMO2025P2] Harmonic AI (2025). "IMO 2025 P2 formalization (Lean)." Summary: Lean formalization file for IMO 2025 Problem 2. URL: https://github.com/harmonic-ai/IMO2025/blob/main/HarmonicLean/IMO2025P2.txt
- [ERDOS-SOTA] Terence Tao et al. (2025). "Erdos Problems" (overview). Summary: Community-maintained status tracker for Erdos problems. URL: https://github.com/teorth/erdosproblemstab=readme-ov-file
- [ERDOS-AI] Terence Tao et al. (2025). "AI contributions to Erdos problems" (wiki). Summary: Notes on AI involvement in Erdos problem progress. URL: https://github.com/teorth/erdosproblems/wiki/AI-contributions-to-Erd%C5%91s-problems
- [LEAN] Lean Prover (2025). "Lean Theorem Prover" (official site). Summary: Official documentation and downloads. URL: https://leanprover.github.io/
